In [5]:
import torch
import torch.nn as nn
from torchvision import models


model = torch.load('./checkpoints/best_validation_model.pt')
# weights = torch.load('./weights/lesion_based_CL_trained_weights/resnet50_128_09.pt')

print(model)

ContrastiveModel(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [7]:
import torch
import torch.nn as nn
from torchvision import models


weights = torch.load('./checkpoints/epoch2_model.pt')
# weights = torch.load('./weights/lesion_based_CL_trained_weights/resnet50_128_09.pt')
print("Done!",type(weights))
model = models.resnet50()
# model = torch.load(weights)
model.load_state_dict(weights, strict=False)
struct = []
for i,j in model.named_parameters():
    struct.append([i,j.shape])
    print(i,j.shape)
with open('b.txt','w') as f:
    for i in struct:
        f.write(str(i))
        f.write('\n')
# weights = torch.load('./checkpoints/best_validation_model.pt')
#model = models.resnet50()
# Our model outputs the score of DR for classification. See https://arxiv.org/pdf/2110.14160.pdf for more details.
#model.fc = nn.Linear(model.fc.in_features, 1)
#model.load_state_dict(weights, strict=True)

Done! <class 'collections.OrderedDict'>
conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
layer1.0.bn3.weight torch.Size([256])
layer1.0.bn3.bias torch.Size([256])
layer1.0.downsample.0.weight torch.Size([256, 64, 1, 1])
layer1.0.downsample.1.weight torch.Size([256])
layer1.0.downsample.1.bias torch.Size([256])
layer1.1.conv1.weight torch.Size([64, 256, 1, 1])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer1.1.conv3.weight torch.Size([256, 64, 1, 1])
layer1.1.bn3.weight torch.Size([256])
layer1.1.bn3.bias tor

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
logits = torch.tensor([[-0.9876,  -1.0815,  -0.2858,  -0.3947],
        [ -0.8289,  -0.5437,  -1.0941,  -0.2471],
        [ -1.4156,  -0.9907,  -0.1231,  -0.6319],
        [ -3.5031,  -1.8569,  -0.6791,  -0.2211]])
logits2 = torch.tensor([[0, 0.5, 0.2, 0.3],
        [0.2, 0, 0.5, 0.3],
        [0.2, 0.4, 0, 0.4],
        [0.1, 0.4, 0.5, 0]])
logits_mask = torch.ones(4)-torch.eye(4)
exp_logits = torch.exp(logits) * logits_mask   #32*32 可能是置信度矩阵，但是没有归一化
        
exp =  torch.exp(logits/2) * logits_mask

# exp_logits = torch.exp(logits)   #32*32 可能是置信度矩阵，但是没有归一化
similar_matrix_t = exp/exp.sum(1,keepdim=True)
# similar_matrix_s = (similar_matrix_t).log()
similar_matrix_s = (similar_matrix_t+torch.eye(4)).log()

print()
print(similar_matrix_t)
print(similar_matrix_s)
criterion2 = nn.KLDivLoss()  
loss = criterion2(similar_matrix_s,similar_matrix_t)
print(loss.item())  #0.018



tensor([[0.0000, 0.2565, 0.3819, 0.3616],
        [0.3112, 0.0000, 0.2725, 0.4163],
        [0.2691, 0.3328, 0.0000, 0.3982],
        [0.1355, 0.3085, 0.5560, 0.0000]])
tensor([[ 0.0000, -1.3606, -0.9627, -1.0172],
        [-1.1673,  0.0000, -1.2999, -0.8764],
        [-1.3128, -1.1003,  0.0000, -0.9209],
        [-1.9990, -1.1759, -0.5870,  0.0000]])
0.0


D:\Python39\lib\site-packages\torch\nn\functional.py:2747: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


In [2]:
import torch
import torch.nn as nn
import resnet
import os
class ContrastiveModel(nn.Module):
    def __init__(self, backbone, pretrained=True, head='mlp', dim_in=2048, feat_dim=128):
        super(ContrastiveModel, self).__init__()

        self.net = backbone(pretrained=True)
        if head == 'linear':
            self.head = nn.Linear(dim_in, feat_dim)
        elif head == 'mlp':
            self.head = nn.Sequential(
                nn.Linear(dim_in, dim_in),
                nn.ReLU(inplace=True),
                nn.Linear(dim_in, feat_dim)
            )


weights = torch.load('./checkpoints_student/best_validation_model.pt')
print("Done!",type(weights))
model = resnet.resnet50()
# model = ContrastiveModel(resnet.resnet50,pretrained=False)
model.load_state_dict(weights, strict=False)
#去掉最后一层
#加入一层
model.classifier = nn.Linear(model.fc.out_features,5)
struct = []
for i,j in model.named_parameters():
    struct.append([i,j.shape])
    print(i,j.shape)
# #保存模型
# save_path = './checkpoints_student/'
# torch.save(model.state_dict(), os.path.join(save_path, 'best_pretrain_model.pt'))
# with open('b.txt','w') as f:
#     for i in struct:
#         f.write(str(i))
#         f.write('\n')

Done! <class 'collections.OrderedDict'>
conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
layer1.0.bn3.weight torch.Size([256])
layer1.0.bn3.bias torch.Size([256])
layer1.0.downsample.0.weight torch.Size([256, 64, 1, 1])
layer1.0.downsample.1.weight torch.Size([256])
layer1.0.downsample.1.bias torch.Size([256])
layer1.1.conv1.weight torch.Size([64, 256, 1, 1])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer1.1.conv3.weight torch.Size([256, 64, 1, 1])
layer1.1.bn3.weight torch.Size([256])
layer1.1.bn3.bias tor

In [5]:
import torch
import torch.nn as nn
import resnet
import os
class ContrastiveModel(nn.Module):
    def __init__(self, backbone, pretrained=True, head='mlp', dim_in=2048, feat_dim=128):
        super(ContrastiveModel, self).__init__()

        self.net = backbone(pretrained=True)
        if head == 'linear':
            self.head = nn.Linear(dim_in, feat_dim)
        elif head == 'mlp':
            self.head = nn.Sequential(
                nn.Linear(dim_in, dim_in),
                nn.ReLU(inplace=True),
                nn.Linear(dim_in, feat_dim)
            )


weights = torch.load('./checkpoints_student/best_validation_model.pt')
print("Done!",type(weights))
model = resnet.resnet50()
# model = ContrastiveModel(resnet.resnet50,pretrained=False)
model.fc = nn.Sequential()   #去掉最后一层fc

model.load_state_dict(weights, strict=False)

# model.fc = nn.Linear(model.fc.in_features,5)
# struct = []
# for i,j in model.named_parameters():
#     struct.append([i,j.shape])
#     print(i,j.shape)
# with open('c.txt','w') as f:
#     for i in struct:
#         f.write(str(i))
#         f.write('\n')
#保存模型
save_path = './checkpoints_student/'
torch.save(model.state_dict(), os.path.join(save_path, 'best_pretrain_model4.pt'))

Done! <class 'collections.OrderedDict'>


In [4]:
import torch
import torch.nn as nn
import resnet
import os

#查看权重
# weights = torch.load('./weights/lesion_based_CL_trained_weights/resnet50_128_08.pt')
weights = torch.load('../Lesion-based_Moco/checkpoints/kaggle_epoch_200.pt')
print("Done!",type(weights))
model = resnet.resnet50()
model.load_state_dict(weights, strict=False)
model.fc = nn.Sequential()   #去掉最后一层fc

struct = []
for i,j in model.named_parameters():
    struct.append([i,j.shape])
    print(i,j.shape)
# with open('our_weight.txt','w') as f:
#     for i in struct:
#         f.write(str(i))
#         f.write('\n')
save_path = './checkpoints_student/'
torch.save(model.state_dict(), os.path.join(save_path, 'pretrain_moco_model_Kaggle_200epoch.pt'))

Done! <class 'collections.OrderedDict'>
conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
layer1.0.bn3.weight torch.Size([256])
layer1.0.bn3.bias torch.Size([256])
layer1.0.downsample.0.weight torch.Size([256, 64, 1, 1])
layer1.0.downsample.1.weight torch.Size([256])
layer1.0.downsample.1.bias torch.Size([256])
layer1.1.conv1.weight torch.Size([64, 256, 1, 1])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer1.1.conv3.weight torch.Size([256, 64, 1, 1])
layer1.1.bn3.weight torch.Size([256])
layer1.1.bn3.bias tor

In [4]:
thresholds = None
a = [-0.5 + i for i in range(5)] if not thresholds else thresholds
print(a)

[-0.5, 0.5, 1.5, 2.5, 3.5]


In [ ]:
class MemoryMoCo(nn.Module):
    """Fixed-size queue with momentum encoder"""
    def __init__(self, feature_dim, queue_size, temperature=0.07):
        super(MemoryMoCo, self).__init__()
        self.queue_size = queue_size
        self.temperature = temperature
        self.index = 0

        # noinspection PyCallingNonCallable
        self.register_buffer('params', torch.tensor([-1]))
        stdv = 1. / math.sqrt(feature_dim / 3)
        memory = torch.rand(self.queue_size, feature_dim, requires_grad=False).mul_(2 * stdv).add_(-stdv)
        self.register_buffer('memory', memory)

    def forward(self, q, k, k_all):
        k = k.detach()

        l_pos = (q * k).sum(dim=-1, keepdim=True)  # shape: (batchSize, 1)
        # TODO: remove clone. need update memory in backwards
        l_neg = torch.mm(q, self.memory.clone().detach().t())
        out = torch.cat((l_pos, l_neg), dim=1)
        out = torch.div(out, self.temperature).contiguous()

        # update memory
        with torch.no_grad():
            all_size = k_all.shape[0]
            out_ids = torch.fmod(torch.arange(all_size, dtype=torch.long).cuda() + self.index, self.queue_size)
            self.memory.index_copy_(0, out_ids, k_all)
            self.index = (self.index + all_size) % self.queue_size

        return out

In [2]:
#数据集抽样
import os,cv2
import random
from tqdm import tqdm

data_path = '/dataset/evaluate/train'
save_path = '/dataset/evaluate_10percent'

for set_name in os.listdir(data_path):
    data_list = []
    set_path = os.path.join(data_path, set_name)
    
    for img in tqdm(os.listdir(set_path)):
        img_path = os.path.join(set_path, img)
        data_list.append(img_path)

    len_data = len(data_list)
    rand_data = random.sample(data_list,0.3*len_data)

    for i in range(len(rand_data)):
        
        path = rand_data[i].split('/')
        write_path = path[-1]
        if i == 0:
            print(save_path+write_path)
        image = cv2.imread(rand_data[i])
        cv2.imwrite(os.path.join(save_path,write_path),image)

print("Done!")

100%|██████████| 25810/25810 [00:00<00:00, 527831.07it/s]


/dataset/evaluate_10percenttrain\0\25527_right.jpeg


100%|██████████| 2443/2443 [00:00<00:00, 492179.48it/s]


/dataset/evaluate_10percenttrain\1\26245_left.jpeg


100%|██████████| 5292/5292 [00:00<00:00, 532080.18it/s]


/dataset/evaluate_10percenttrain\2\155_right.jpeg


100%|██████████| 873/873 [00:00<00:00, 496289.97it/s]


/dataset/evaluate_10percenttrain\3\32311_left.jpeg


100%|██████████| 708/708 [00:00<00:00, 355168.91it/s]


/dataset/evaluate_10percenttrain\4\6017_right.jpeg
Done!


In [22]:
import torch
import torch.nn.functional as F
a = torch.tensor([[-1.1448, -2.4910, -2.2625, -2.2295, -2.3152, -2.3273, -2.0577, -2.7621],
        [-2.4162, -1.0701, -2.3490, -2.4088, -2.2759, -2.3028, -2.5752, -2.2678],
        [-2.2947, -2.4560, -1.1770, -2.1975, -2.5606, -2.1604, -2.1830, -2.4203],
        [-2.2196, -2.4737, -2.1555, -1.1350, -2.5405, -2.4265, -2.2433, -2.3380],
        [-2.2337, -2.2692, -2.4470, -2.4688, -1.0634, -2.5249, -2.4462, -2.2420],
        [-2.3279, -2.3781, -2.1288, -2.4369, -2.6069, -1.1454, -2.3120, -2.1855],
        [-2.0590, -2.6513, -2.1521, -2.2544, -2.5289, -2.3127, -1.1461, -2.4760],
        [-2.7208, -2.3012, -2.3468, -2.3066, -2.2821, -2.1436, -2.4333, -1.1035]])
b = torch.tensor([[0.2965, 0.0645, 0.1034, 0.0967, 0.1360, 0.0819, 0.1212, 0.0997],
        [0.0759, 0.3489, 0.0827, 0.0670, 0.0939, 0.1156, 0.1054, 0.1105],
        [0.1090, 0.0741, 0.3125, 0.0802, 0.1084, 0.1010, 0.1199, 0.0948],
        [0.1047, 0.0617, 0.0824, 0.3210, 0.1104, 0.0956, 0.1096, 0.1146],
        [0.1283, 0.0753, 0.0971, 0.0963, 0.2798, 0.1075, 0.1187, 0.0970],
        [0.0856, 0.1027, 0.1002, 0.0923, 0.1191, 0.3100, 0.0981, 0.0919],
        [0.1156, 0.0854, 0.1086, 0.0966, 0.1201, 0.0896, 0.2830, 0.1011],
        [0.1015, 0.0956, 0.0915, 0.1077, 0.1046, 0.0895, 0.1078, 0.3018]])
a = torch.exp(a)
loss = F.kl_div(a,b,reduction='batchmean')
print(a,'\n',b)
print(loss)

tensor([[0.3183, 0.0828, 0.1041, 0.1076, 0.0987, 0.0976, 0.1277, 0.0632],
        [0.0893, 0.3430, 0.0955, 0.0899, 0.1027, 0.1000, 0.0761, 0.1035],
        [0.1008, 0.0858, 0.3082, 0.1111, 0.0773, 0.1153, 0.1127, 0.0889],
        [0.1087, 0.0843, 0.1158, 0.3214, 0.0788, 0.0883, 0.1061, 0.0965],
        [0.1071, 0.1034, 0.0866, 0.0847, 0.3453, 0.0801, 0.0866, 0.1062],
        [0.0975, 0.0927, 0.1190, 0.0874, 0.0738, 0.3181, 0.0991, 0.1124],
        [0.1276, 0.0706, 0.1162, 0.1049, 0.0797, 0.0990, 0.3179, 0.0841],
        [0.0658, 0.1001, 0.0957, 0.0996, 0.1021, 0.1172, 0.0877, 0.3317]]) 
 tensor([[0.2965, 0.0645, 0.1034, 0.0967, 0.1360, 0.0819, 0.1212, 0.0997],
        [0.0759, 0.3489, 0.0827, 0.0670, 0.0939, 0.1156, 0.1054, 0.1105],
        [0.1090, 0.0741, 0.3125, 0.0802, 0.1084, 0.1010, 0.1199, 0.0948],
        [0.1047, 0.0617, 0.0824, 0.3210, 0.1104, 0.0956, 0.1096, 0.1146],
        [0.1283, 0.0753, 0.0971, 0.0963, 0.2798, 0.1075, 0.1187, 0.0970],
        [0.0856, 0.1027, 0.1002, 0.